In [7]:
import pandas as pd
import numpy as np

In [8]:
link_df = pd.read_csv("links.csv", index_col=0)
link_df.head()

,from,to,value,from_id,to_id
0,慶應義塾大学,研究,0.025952,4,1405
1,慶應義塾大学,学部,0.022282,4,439
2,慶應義塾大学,月,0.020959,4,2603
3,慶應義塾大学,学生,0.020782,4,1603
4,慶應義塾大学,情報,0.019950,4,2291


In [9]:
node_df = pd.read_csv("nodes.csv", index_col=0)
node_df.head(6)

,query,from_ut,from_ky,from_wa,from_ko,from_me,shortest_step,label_ut,label_ky,label_wa,label_ko,label_me,index
0,東京大学,0,2,2,2,2,0,True,False,False,False,False,0
1,京都大学,2,0,2,2,2,0,False,True,False,False,False,1
2,明治大学,2,2,2,2,0,0,False,False,False,False,True,2
3,早稲田大学,2,2,0,2,2,0,False,False,True,False,False,3
4,慶應義塾大学,2,2,2,0,2,0,False,False,False,True,False,4
5,PDF,2,2,3,3,2,2,True,True,False,False,True,5


In [10]:
node_df[node_df.shortest_step == 0]

,query,from_ut,from_ky,from_wa,from_ko,from_me,shortest_step,label_ut,label_ky,label_wa,label_ko,label_me,index
0,東京大学,0,2,2,2,2,0,True,False,False,False,False,0
1,京都大学,2,0,2,2,2,0,False,True,False,False,False,1
2,明治大学,2,2,2,2,0,0,False,False,False,False,True,2
3,早稲田大学,2,2,0,2,2,0,False,False,True,False,False,3
4,慶應義塾大学,2,2,2,0,2,0,False,False,False,True,False,4


In [14]:
node_df.columns

Index(['query', 'from_ut', 'from_ky', 'from_wa', 'from_ko', 'from_me',
       'shortest_step', 'label_ut', 'label_ky', 'label_wa', 'label_ko',
       'label_me', 'index'],
      dtype='object')

In [11]:
node_ids = node_df['index'].values

In [29]:
nodes_data = node_df[['from_ut', 'from_ky', 'from_wa', 'from_ko', 'from_me']].values

In [51]:
nearest_univ_nodes = [node_data.argmin() for node_data in nodes_data[5:]]

In [53]:
from numpy.random import *
positions = []
for nearest_univ_node in nearest_univ_nodes:
    if nearest_univ_node == 0:
        p = np.array([uniform(-3, -2, 1)[0], uniform(2, 3, 1)[0], uniform(2, 3, 1)[0]])
    elif nearest_univ_node == 1:
        p = uniform(-3, -2, 3)
    elif nearest_univ_node == 2:
        p = np.array([uniform(2, 3, 1)[0], uniform(-3, -2, 1)[0], uniform(2, 3, 1)[0]])
    elif nearest_univ_node == 3:
        p = np.array([uniform(-3, -2, 1)[0], uniform(2, 3, 1)[0], uniform(-3, -2, 1)[0]])
    elif nearest_univ_node == 4:
        p = np.array([uniform(2, 3, 1)[0], uniform(2, 3, 1)[0], uniform(-3, -2, 1)[0]])
    positions.append(p)
not_univ_nodes_p = np.array(positions)

In [60]:
univs_positions = np.zeros((5, 3))
univs_positions[0] = np.array([-1,1,1])
univs_positions[1] = np.array([-1,-1,-1])
univs_positions[2] = np.array([1,1,-1])
univs_positions[3] = np.array([1,-1,1])
univs_positions[4] = np.array([-1,1,-1])

In [61]:
start_positions = np.vstack((univs_positions, not_univ_nodes_p))

In [6]:
# from_ids =  np.array(link_df.from_id)
# to_ids = np.array(link_df.to_id)
# distances = np.array(node_df.shortest_step)
# from numpy.random import *
# start_positions = uniform(-2,2, (len(node_ids), 3))
# start_positions[0] = np.array([-1,1,1])
# start_positions[1] = np.array([-1,-1,-1])
# start_positions[2] = np.array([1,1,-1])
# start_positions[3] = np.array([1,-1,1])
# start_positions[4] = np.array([-1,1,-1])
# import copy
# positions = copy.deepcopy(start_positions)

In [69]:
# 重心法
import copy
positions = copy.deepcopy(start_positions)
for i in range(500): 
    Fs = [0]* len(node_ids)
    for node_id in node_ids:
        if node_id not in [0, 1, 2, 3, 4]:
            P = positions[node_id]
            F = np.array([0, 0, 0])
            linked_node_ids = to_ids[np.where(from_ids == node_id)]
            for node_i_id in linked_node_ids:
                D_i = float(distances[node_i_id] + 1)
                P_i = positions[node_i_id]
                x_i = (P_i - P) - ((P_i - P) / np.linalg.norm(P_i - P)) # バネの伸びベクトル(バネのもとの長さは1)
                F_i = x_i * (1/D_i)  # Nに働く力ベクトルを算出(バネ定数はD_iの逆数)
                F = F + F_i
            Fs[node_id] =F / len(linked_node_ids)
    for node_id in node_ids:
        positions[node_id] = positions[node_id] + Fs[node_id]
    if i % 10 == 0:
        print(np.sum(np.absolute(np.array(Fs[5:]))))

1140.7899356938567
158.91895627483052


KeyboardInterrupt: 

In [67]:
# takeuchi
import copy
positions = copy.deepcopy(start_positions)
for i in range(500): 
    Fs = [0]* len(node_ids)
    for node_id in node_ids:
        if node_id not in [0, 1, 2, 3, 4]:
            P = positions[node_id]
            F = np.array([0, 0, 0])
            linked_node_ids = to_ids[np.where(from_ids == node_id)]
            for node_i_id in linked_node_ids:
                D_i = float(distances[node_i_id] + 1)
                P_i = positions[node_i_id]
                x_i = (P_i - P) - ((P_i - P) / np.linalg.norm(P_i - P)) # バネの伸びベクトル(バネのもとの長さは1)
                F_i = x_i * (1/D_i)  # Nに働く力ベクトルを算出(バネ定数はD_iの逆数)
                F = F + F_i
            Fs[node_id] = F * D_i * 0.001
    for node_id in node_ids:
        positions[node_id] = positions[node_id] + Fs[node_id]
    if i % 10 == 0:
        print(np.sum(np.absolute(np.array(Fs[5:]))))

48.485859719452286
32.49836392939171
23.7932050257033
18.39887680284659
14.814797752635707
12.331077621041269
10.560151035954886
9.242227427327007
8.239636341735821
7.461290909544442
6.84799022427804
6.354325528555847
5.949979208672138
5.613248075642769
5.329277734971424
5.088125056760351
4.881611735984054
4.700002098362922
4.537864585049606
4.393138055597275
4.263312844229639
4.146176710295102
4.039973560777379
3.9431444270343206
3.8532974756037337
3.77017014608897
3.693007650732934
3.6209997004027272
3.5538528339490423
3.491097427095971
3.4326642104905916
3.3775452317415997
3.3255027119023644
3.275873305265272
3.2283116825706366
3.18258989661263
3.139253159586682
3.0976532174903637
3.0575804731798852
3.019103189852526
2.9819101407054367
2.945944258372688
2.91105696303956
2.8771358209805773
2.844215984403352
2.8122211644418917
2.7812640878053996
2.751372805887579
2.722232776167408
2.6939580751540637


In [68]:
# pd.DataFrame(positions, columns=['x','y','z']).to_csv('normal2.csv')
pd.DataFrame(positions, columns=['x','y','z']).to_csv('takeuchi2.csv')

In [179]:
start_positions

array([[-1.        ,  1.        ,  1.        ],
       [-1.        , -1.        , -1.        ],
       [ 1.        ,  1.        , -1.        ],
       ...,
       [ 0.6928154 ,  1.68818213, -0.9419346 ],
       [ 0.66579461, -0.43664928, -1.49406485],
       [ 1.44154426,  1.92945304, -0.58298092]])

In [180]:
positions

array([[-1.        ,  1.        ,  1.        ],
       [-1.        , -1.        , -1.        ],
       [ 1.        ,  1.        , -1.        ],
       ...,
       [ 0.36739009,  1.24925322, -0.72338481],
       [ 0.45182163, -0.13944754, -0.7450748 ],
       [ 0.05837693,  0.54831132, -0.02332937]])